In [1]:
import os

In [2]:
%pwd

'e:\\DataScienceProjects\\Ml_Project_with_mlflow\\research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL:str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from src.mlproject.constants import *
from src.mlproject.utils.common import *

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH) :
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_files,
            unzip_dir=config.unzip_dir

        )
        return data_ingestion_config        

In [7]:
import os
import urllib.request as request
import zipfile
from src.mlproject import logger
from src.mlproject.utils.common import get_size


In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config=config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers=request.urlretrieve(
            url=self.config.source_URL,
            filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [16]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2023-08-15 23:27:24,403: INFO: common: yaml File: config\config.yaml loaded successfully]
[2023-08-15 23:27:24,403: INFO: common: yaml File: params.yaml loaded successfully]
[2023-08-15 23:27:24,412: INFO: common: yaml File: schema.yaml loaded successfully]
[2023-08-15 23:27:24,412: INFO: common: Created directory at artifacts]
[2023-08-15 23:27:24,412: INFO: common: Created directory at artifacts/data_ingestion]


[2023-08-15 23:27:25,486: INFO: 2202453773: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 5300:3B34A1:82DC1:C7725:64DBBC83
Accept-Ranges: bytes
Date: Tue, 15 Aug 2023 17:57:24 GMT
Via: 1.1 varnish
X-Served-By: cache-del21728-DEL
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1692122245.542845,VS0,VE317
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: fb16f2acee0b4fcfc7285148e8a49a989db3fce9
Expires: Tue, 15 Aug 2023 18:02:24 GMT
Source-Age: 0

]
